In [1]:
import json
import pandas as pd
import re
import os
from collections import Counter
from pathlib import Path
from pathlib import Path

In [2]:
# Baca data dulu ke dalam df
df = pd.read_csv("D:\SEMESTER 6\PROJECT CBR\putusan_ma__2025-06-23.csv") 

<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
C:\Users\ASUS\AppData\Local\Temp\ipykernel_6772\1433169407.py:2: SyntaxWarning: invalid escape sequence '\S'
  df = pd.read_csv("D:\SEMESTER 6\PROJECT CBR\putusan_ma__2025-06-23.csv")


In [3]:
# Fungsi Ringkasan Fakta
def ringkasan_fakta(teks):
    kalimat = str(teks).split('. ')
    return '. '.join(kalimat[:2]).strip()

# Fungsi Argumen Hukum
def argumen_hukum(teks):
    kalimat = [k for k in str(teks).split('. ') if 'pasal' in k.lower()]
    return '. '.join(kalimat[:2]).strip()

# Jumlah kata
def jumlah_kata(teks):
    return len(str(teks).split())

# Bag-of-Words (ambil top 5 kata paling sering)
def bag_of_words(teks, top_n=5):
    words = re.findall(r'\b\w+\b', str(teks).lower())
    stopwords = {'yang', 'dan', 'di', 'ke', 'dengan', 'untuk', 'dari', 'pada', 'adalah'}
    words = [w for w in words if w not in stopwords]
    common = Counter(words).most_common(top_n)
    return ', '.join([w for w, _ in common])

# QA-pairs sederhana
def qa_pair(fakta, amar):
    if not isinstance(fakta, str) or not isinstance(amar, str):
        return "-"
    tanya = f"Apa yang terjadi dalam kasus ini? {fakta}"
    jawab = f"Apa hasil putusannya? {amar}"
    return f"Q: {tanya}\nA: {jawab}"

# Tambahkan kolom-kolom baru
df['case_id'] = df.index + 1
df['ringkasan_fakta'] = df['text_pdf'].apply(ringkasan_fakta)
df['argumen_hukum'] = df['text_pdf'].apply(argumen_hukum)
df['text_length'] = df['text_pdf'].apply(jumlah_kata)
df['bag_of_words'] = df['text_pdf'].apply(bag_of_words)
df['qa_pair'] = df.apply(lambda x: qa_pair(x['ringkasan_fakta'], x['amar']), axis=1)

# Pilih kolom akhir yang ingin disimpan
cols = ['case_id', 'nomor', 'tanggal_register', 'klasifikasi', 'ringkasan_fakta',
        'argumen_hukum', 'amar', 'hakim_ketua', 'text_pdf', 'text_length', 'bag_of_words', 'qa_pair']

# Buat folder 'data/processed' jika belum ada
Path(r"D:/SEMESTER 6/PROJECT CBR/data/processed").mkdir(parents=True, exist_ok=True)

# Simpan ke CSV
df[cols].to_csv("D:/SEMESTER 6/PROJECT CBR/data/processed/cases.csv", index=False, encoding='utf-8')

# Simpan juga ke JSON
df[cols].to_json("D:/SEMESTER 6/PROJECT CBR/data/processed/cases.json", orient='records', force_ascii=False, indent=2)